In [ ]:
import os
import re
import ast
import textwrap
from pathlib import Path
from dataclasses import dataclass
from typing import List, Protocol, Optional
from openai import BadRequestError as _OpenAIBadRequest
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


load_dotenv(override=True)

TESTGEN_MODEL = os.getenv("TESTGEN_MODEL", "gpt-5-nano")
OPENAI_CLIENT = OpenAI()

In [ ]:

class CompletionClient(Protocol):
    """Any LLM client provides a .complete() method."""
    def complete(self, *, model: str, system: str, user: str) -> str: ...


class OpenAIChatClient:
    """Adapter over OpenAI chat completions that omits unsupported params."""
    def __init__(self, client: OpenAI):
        self._client = client

    def _call(self, *, model: str, system: str, user: str, include_temperature: bool = False) -> str:
        params = {
            "model": model,
            "messages": [
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
        }
        if include_temperature:  
            params["temperature"] = 0.7 

        resp = self._client.chat.completions.create(**params)
        text = (resp.choices[0].message.content or "").strip()
        return _extract_code_or_text(text)

    def complete(self, *, model: str, system: str, user: str) -> str:
        try:
            return self._call(model=model, system=system, user=user, include_temperature=False)
        except _OpenAIBadRequest as e:
            # Extra safety: if some lib auto-injected temperature, retry without it
            if "temperature" in str(e).lower():
                return self._call(model=model, system=system, user=user, include_temperature=False)
            raise


In [ ]:
def _extract_code_or_text(s: str) -> str:
    """Prefer fenced python if present; otherwise return raw text."""
    m = re.search(r"```(?:python)?\s*(.*?)```", s, flags=re.S | re.I)
    return m.group(1).strip() if m else s.strip()


def _ensure_header_and_import(code: str, module_name: str) -> str:
    """Make sure tests import the module and pytest; keep output minimal."""
    code = code.strip()
    needs_pytest = "import pytest" not in code
    needs_import = f"import {module_name}" not in code and f"import {module_name} as mod" not in code

    header_lines = []
    if needs_pytest:
        header_lines.append("import pytest")
    if needs_import:
        header_lines.append(f"import {module_name} as mod")

    if header_lines:
        code = "\n".join(header_lines) + "\n\n" + code
    return code


In [ ]:
@dataclass(frozen=True)
class SymbolInfo:
    kind: str      # "function" | "class" | "method"
    name: str
    signature: str
    lineno: int


class PublicAPIExtractor:
    """Extract a small 'public API' summary from a Python module."""
    def extract(self, source: str) -> List[SymbolInfo]:
        tree = ast.parse(source)
        out: List[SymbolInfo] = []

        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and not node.name.startswith("_"):
                out.append(SymbolInfo("function", node.name, self._sig(node), node.lineno))
            elif isinstance(node, ast.ClassDef) and not node.name.startswith("_"):
                out.append(SymbolInfo("class", node.name, node.name, node.lineno))
                for sub in node.body:
                    if isinstance(sub, ast.FunctionDef) and not sub.name.startswith("_"):
                        out.append(SymbolInfo("method",
                                              f"{node.name}.{sub.name}",
                                              self._sig(sub),
                                              sub.lineno))
        return sorted(out, key=lambda s: (s.kind, s.name.lower(), s.lineno))

    def _sig(self, fn: ast.FunctionDef) -> str:
        args = [a.arg for a in fn.args.args]
        if fn.args.vararg:
            args.append("*" + fn.args.vararg.arg)
        args.extend(a.arg + "=?" for a in fn.args.kwonlyargs)
        if fn.args.kwarg:
            args.append("**" + fn.args.kwarg.arg)
        ret = ""
        if fn.returns is not None:
            try:
                ret = f" -> {ast.unparse(fn.returns)}"
            except Exception:
                pass
        return f"def {fn.name}({', '.join(args)}){ret}:"


In [ ]:
class PromptBuilder:
    """Builds concise, deterministic prompts for pytest generation."""
    SYSTEM = (
        "You are a senior Python engineer. Produce a single, self-contained pytest file.\n"
        "Rules:\n"
        "- Output only Python test code (no prose, no markdown fences).\n"
        "- Use plain pytest tests (functions), no classes unless unavoidable.\n"
        "- Deterministic: avoid network/IO; seed randomness if used.\n"
        "- Import the target module by module name.\n"
        "- Create a minimal test covering every public function and method.\n"
        "- Prefer straightforward, fast assertions over exhaustive checks.\n"
    )

    def build_user(self, *, module_name: str, source: str, symbols: List[SymbolInfo]) -> str:
        summary = "\n".join(f"- {s.kind:<6}  {s.signature}" for s in symbols) or "- (no public symbols)"
        return textwrap.dedent(f"""
        Create pytest tests for module `{module_name}`.

        Public API Summary:
        {summary}

        Constraints:
        - Import as: `import {module_name} as mod`
        - Keep tests tiny, fast, and deterministic.

        Full module source (for reference):
        # --- BEGIN SOURCE {module_name}.py ---
        {source}
        # --- END SOURCE ---
        """).strip()


In [ ]:
class TestGenerator:
    """Orchestrates extraction, prompt, model call, and final polish."""
    def __init__(self, llm: CompletionClient, model: str):
        self._llm = llm
        self._model = model
        self._extractor = PublicAPIExtractor()
        self._prompts = PromptBuilder()

    def generate_tests(self, module_name: str, source: str) -> str:
        symbols = self._extractor.extract(source)
        user = self._prompts.build_user(module_name=module_name, source=source, symbols=symbols)
        raw = self._llm.complete(model=self._model, system=self._prompts.SYSTEM, user=user)
        return _ensure_header_and_import(raw, module_name)


In [ ]:
LLM = OpenAIChatClient(OPENAI_CLIENT)
SERVICE = TestGenerator(LLM, TESTGEN_MODEL)

def build_module_name_from_path(path: str) -> str:
    p = Path(path)
    return p.stem

def generate_from_code(module_name: str, code: str, save: bool, out_dir: str) -> tuple[str, str]:
    if not module_name.strip():
        return "", "❌ Please provide a module name."
    if not code.strip():
        return "", "❌ Please paste some Python code."

    tests_code = SERVICE.generate_tests(module_name=module_name.strip(), source=code)
    saved = ""
    if save:
        out = Path(out_dir or "tests")
        out.mkdir(parents=True, exist_ok=True)
        out_path = out / f"test_{module_name}.py"
        out_path.write_text(tests_code, encoding="utf-8")
        saved = f"✅ Saved to {out_path}"
    return tests_code, saved


def generate_from_file(file_obj, save: bool, out_dir: str) -> tuple[str, str]:
    if file_obj is None:
        return "", "❌ Please upload a .py file."
    code = file_obj.decode("utf-8")
    module_name = build_module_name_from_path("uploaded_module.py")
    return generate_from_code(module_name, code, save, out_dir)


In [ ]:
with gr.Blocks(title="Simple PyTest Generator") as ui:
    gr.Markdown("## 🧪 Simple PyTest Generator (Week 4 • Community Contribution)\n"
                "Generate **minimal, deterministic** pytest tests from a Python module using a Frontier model.")

    with gr.Tab("Paste Code"):
        with gr.Row():
            module_name = gr.Textbox(label="Module name (used in `import <name> as mod`)", value="mymodule")
        code_in = gr.Code(label="Python module code", language="python", lines=22)
        with gr.Row():
            save_cb = gr.Checkbox(label="Save to /tests", value=True)
            out_dir = gr.Textbox(label="Output folder", value="tests")
        gen_btn = gr.Button("Generate tests", variant="primary")
        with gr.Row():
            tests_out = gr.Code(label="Generated tests (pytest)", language="python", lines=20)
        status = gr.Markdown()

        def _on_gen(name, code, save, outdir):
            tests, msg = generate_from_code(name, code, save, outdir)
            return tests, (msg or "✅ Done")

        gen_btn.click(_on_gen, inputs=[module_name, code_in, save_cb, out_dir], outputs=[tests_out, status])

    with gr.Tab("Upload .py"):
        upload = gr.File(file_types=[".py"], label="Upload a Python module")
        with gr.Row():
            save_cb2 = gr.Checkbox(label="Save to /tests", value=True)
            out_dir2 = gr.Textbox(label="Output folder", value="tests")
        gen_btn2 = gr.Button("Generate tests from file")
        tests_out2 = gr.Code(label="Generated tests (pytest)", language="python", lines=20)
        status2 = gr.Markdown()

        def _on_gen_file(f, save, outdir):
            tests, msg = generate_from_file(f.read() if f else None, save, outdir)
            return tests, (msg or "✅ Done")

        gen_btn2.click(_on_gen_file, inputs=[upload, save_cb2, out_dir2], outputs=[tests_out2, status2])

ui.launch(inbrowser=True)
